In [1]:
import sys
import os
# Get the parent directory path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

Load the images names

In [2]:
images = []
dataDir = "../images1/"
for img in os.listdir(dataDir):

    #join the path with the names of the images
    dataDirTemp = os.path.join(dataDir, str(img))
    images.append(dataDirTemp)
print("Images to be processed: ", images)

Images to be processed:  ['../images1/G000_IMG062.jpg', '../images1/G000_IMG087.jpg', '../images1/G000_IMG102.jpg', '../images1/G006_IMG048.jpg', '../images1/G006_IMG086.jpg', '../images1/G006_IMG119.jpg', '../images1/G019_IMG082.jpg', '../images1/G028_IMG015.jpg', '../images1/G028_IMG062.jpg', '../images1/G028_IMG098.jpg', '../images1/G028_IMG101.jpg', '../images1/G033_IMG043.jpg', '../images1/G033_IMG075.jpg', '../images1/G033_IMG088.jpg', '../images1/G033_IMG101.jpg', '../images1/G038_IMG074.jpg', '../images1/G038_IMG088.jpg', '../images1/G038_IMG103.jpg', '../images1/G038_IMG105.jpg', '../images1/G041_IMG042.jpg', '../images1/G041_IMG048.jpg', '../images1/G041_IMG088.jpg', '../images1/G041_IMG098.jpg', '../images1/G047_IMG053.jpg', '../images1/G047_IMG068.jpg', '../images1/G047_IMG102.jpg', '../images1/G047_IMG107.jpg', '../images1/G056_IMG017.jpg', '../images1/G056_IMG077.jpg', '../images1/G056_IMG097.jpg', '../images1/G058_IMG044.jpg', '../images1/G058_IMG074.jpg', '../images1/G0

In [ ]:
import cv2
import numpy as np
def orient_board_by_logo(warped_board,debug=False):
    # Define a percentage of the board size for the corner region
    height, width = warped_board.shape[:2]
    corner_h, corner_w = 30,30
    offset_h, offset_w = 10, 20

    # Convert to grayscale for intensity calculation
    if len(warped_board.shape) > 2:
        gray_board = cv2.cvtColor(warped_board, cv2.COLOR_BGR2GRAY)
        vis_board = warped_board.copy()

    else:
        gray_board = warped_board
        vis_board = cv2.cvtColor(warped_board, cv2.COLOR_GRAY2BGR)


 


    # Define the four corner ROIs (Regions of Interest)
    corners = {
        "top_left": gray_board[offset_h:corner_h + offset_h, offset_w:corner_w + offset_w],
        "top_right": gray_board[offset_h:corner_h + offset_h, width-1 -corner_w - offset_w:width- 1- offset_w],
        "bottom_left": gray_board[height-1 -corner_h:height-1 , offset_w:corner_w + offset_w],
        "bottom_right": gray_board[height-1 -corner_h:height-1 , width-1 -corner_w:width-1 ]
    }
    cv2.imshow("top_left", corners["top_left"])
    cv2.imshow("top_right", corners["top_right"])
    cv2.imshow("bottom_left", corners["bottom_left"])
    cv2.imshow("bottom_right", corners["bottom_right"])
    # Calculate the average intensity of each corner
    avg_intensities = {name: np.mean(roi) for name, roi in corners.items()}

    # The logo corner will be the darkest (lowest intensity)
    logo_corner = min(avg_intensities, key=avg_intensities.get)

    # Determine rotation needed to move the logo to the bottom-left
    if logo_corner == "bottom_left":
        rotation_code = 0
        oriented_board = warped_board
        cv2.rectangle(vis_board, (offset_w, height-1 -corner_h), (corner_w + offset_w, height-1), (255, 0, 0), 2)  # bottom-left
    elif logo_corner == "bottom_right":
        rotation_code = 90  
        oriented_board = cv2.rotate(warped_board, cv2.ROTATE_90_CLOCKWISE)
        cv2.rectangle(vis_board, (width-1 -corner_w, height-1 -corner_h), ( width-1, height-1), (0, 255, 255), 2)  # bottom-right
    elif logo_corner == "top_right":
        rotation_code = 180  
        oriented_board = cv2.rotate(warped_board, cv2.ROTATE_180)
        cv2.rectangle(vis_board, (width-corner_w, offset_h), (width, corner_h+offset_h), (0, 255, 0), 2)  # top-right
    elif logo_corner == "top_left":
        rotation_code = 270 
        oriented_board = cv2.rotate(warped_board, cv2.ROTATE_90_COUNTERCLOCKWISE)
        cv2.rectangle(vis_board, (offset_w, offset_h), (corner_w+offset_w, corner_h+offset_h), (0, 0, 255), 2)  # top-left
    else:
        rotation_code = 0
        oriented_board = warped_board
    cv2.imshow("Corner Highlight Visualization", vis_board)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return oriented_board, rotation_code

def rotate_presence_matrix(matrix, angle):
    if not matrix or not matrix[0]:
        return matrix
        
    if angle == 90:
        return [list(row) for row in zip(*matrix[::-1])]
    elif angle == 180:
        return [row[::-1] for row in matrix[::-1]]
    elif angle == 270:
        return [list(row) for row in zip(*matrix)][::-1]
    return matrix

In [18]:
from task3_2API import detectBoardAndSquares
for img in images[:1]:
    print("Processing image: ", img)
    # Call the function to detect board and squares
    normalizedBoard, square_box, M, fx, fy = detectBoardAndSquares(img)
    if(normalizedBoard is not None):
        orient_board_by_logo(normalizedBoard)
    print("Finished processing image: ", img)

Processing image:  ../images1/G000_IMG062.jpg
Chessboard found in ../images1/G000_IMG062.jpg
All squares found
Finished processing image:  ../images1/G000_IMG062.jpg
